Uncomment the cell below if you dont have libraries

In [ ]:
#%pip install pandas numpy scikit-learn==0.21.3 pyltr

In [ ]:
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import StandardScaler
import pyltr
import pickle

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive/', force_remount=True)

In [ ]:
#base_path = os.path.join('/content/drive/MyDrive/Colab Notebooks/dmt_asg_2/')
base_path = os.getcwd()

training_set_file_path = os.path.join(base_path,'data/train_data_rf.csv')
test_set_file_path = os.path.join(base_path,'data/test_data_rf.csv')
training_data = pd.read_csv(training_set_file_path)
test_data = pd.read_csv(test_set_file_path)

In [ ]:
training_data_y = training_data.relevancy
search_id = training_data.srch_id.tolist()
search_id_test = test_data.srch_id.tolist()
prop_id_test = test_data.prop_id.tolist()
splitNumber = int(len(training_data_y)*0.9)

In [ ]:
training_data_x = training_data.drop(['srch_id','relevancy'],axis=1)
test_data = test_data.drop(['srch_id','prop_id'],axis=1)

In [ ]:
val_data_x = training_data_x[splitNumber:]
val_data_y = training_data_y[splitNumber:]
val_search_id = search_id[splitNumber:]
training_data_x = training_data_x[:splitNumber]
training_data_y = training_data_y[:splitNumber]
training_search_id = search_id[:splitNumber]

In [ ]:
scaler = StandardScaler().fit(training_data_x)
training_data_x = scaler.transform(training_data_x)
scaler = StandardScaler().fit(val_data_x)
val_data_x = scaler.transform(val_data_x)
scaler = StandardScaler().fit(test_data)
test_data = scaler.transform(test_data)

In [ ]:
metric = pyltr.metrics.NDCG(k=5)

monitor = pyltr.models.monitors.ValidationMonitor(val_data_x, val_data_y, val_search_id, metric=metric,
                                                      stop_after=10)
model = pyltr.models.LambdaMART(
    metric=metric,
    n_estimators=400,
    max_features=0.5,
    learning_rate=0.02,
    query_subsample=0.5,
    max_leaf_nodes=10,
    min_samples_leaf=64,
    verbose=1,
)

model.fit(training_data_x, training_data_y, training_search_id, monitor=monitor)

In [ ]:
pickle.dump(model, open(os.path.join(base_path,'finalized_model.sav'), 'wb'))

In [ ]:
loaded_model = pickle.load(open(os.path.join(base_path,'finalized_model.sav'), 'rb'))

In [ ]:
predictions = model.predict(test_data)

In [ ]:
outputFrame = pd.DataFrame(columns=['srch_id','predictions', 'prop_id'])
outputFrame.srch_id = search_id_test
outputFrame.predictions = predictions
outputFrame.prop_id = prop_id_test
outputFrame = outputFrame.sort_values(['srch_id', 'predictions'], ascending = [True, False])
outputFrame = outputFrame.drop(['predictions'],1)

In [ ]:
out_path = os.path.join(base_path,'data/lambdaTest.csv')
outputFrame.to_csv(path_or_buf=out_path, index=False)